# **Idealista scraper**
*Jose Ramon Estevez Melgarejo 2022-03-04.*

## Introduction
Wellcome to the first mini project of my personal portfolio where I create a web scraper to obtain valuable informmation from [Idealista](https://www.idealista.com). When structuring my portfolio I thought that a good Exploratory Data Analysis (EDA) was the first thing I needed to do in order to showcase my abilities. After searching for a messy dataset I could work with for some time I decided to create my own scraper to obtain data for my analysis. So this project will be divided into tree parts:

1.  Data extraction via web scraping (current notebook) 
2.  Exploratory Data Analysis (EDA)
3.  Machine Learning Regression model to estimate huse price

### Why idealista?
Well idealista is one of the main real state platforms that spanish people use to sell and buy houses. After having worked as a data scientist for more than three years I have finished plenty of projects but it is difficult to use them for my portfolio whithout revealing sensitive informmation about the companies I have been working for. Therefore, I decided tu develop a study about the real state situation at my home city (Cadiz) that I could maybe benefit from. Also, idealista web site is a relitively easy site to scrap and data is not really well structured (I wanted to start my EDA with messy data as data cleanishing is also a big part of data science).


### Scraping method
As far as I know there are three main scraping libraries / frameworks. Scrapy, Selenium and Requests + Beautifulsoup. Eventhough Scrapy and selenium seem to be more robust alternatives I decided not to overcomplicate  my self and use Requests + Beautifulsou given that idealista web site is relatively simple to scrap.

## Index
1.  Importing Libraries & script genral variables
2.  Scraping free proxies and defining Request Headers
3.  Houses scraping
    1.  Houses ids scraping
    2.  Houses info scraping



## 1. Importing Libraries 

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import proxies_scr # source to a personal script to scrap free proxy sites

First we will define two variables that will help us to limit our house search if we wanted to and select the number of free working proxies that  we will use to scrpa our data. why we use free proxies will be explained in the next section. 

In [ ]:
search_limit = 10 # Limit search of houses.
n_prox = 7 # Selecting the number of free proxies to use.

## 2.  Scraping free proxies and defining Request Headers

It is very common that websites block your IP if you do many consecutive requests in a short period of time or if they suspect that one entering the site is not a human directly (our case). So in order to avoid been blocked we have to do two things. prueba git

In [2]:
# We need to send headers so that idealista recognises us as a person and not a bot
headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es,es-ES;q=0.9,en;q=0.8,fr;q=0.7',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'referer': 'https://www.idealista.com/en/areas/venta-viviendas/?shape=%28%28ez_%7EEn%7Bse%40_bCceIniKcdFpvAfiEa%7EI%7E_J%29%29',
    #'sec-ch-ua': " Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98",
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': 'macO',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36',
    'x-newrelic-id': 'VQIGUlZbGwIBXFhWBQEDVw==',
    'x-requested-with': 'XMLHttpRequest'
}

In [3]:
# Looking  for proxies
print('Selecting working proxies.')
working_proxies = []
tested_proxies = []
round = 1
n_prox = 1

while len(working_proxies) < n_prox:
    print(f'Round {round} of proxy scrapping.')
    print(f'{len(tested_proxies)} proxies tested')
    print(f'{len(working_proxies)} proxies found')
    proxies = proxies_scr.get_proxies()
    for prox in proxies:
        if prox not in working_proxies and prox not in tested_proxies:
            if len(working_proxies) >= n_prox:
                break
            else:

                try:
                    url = 'https://www.idealista.com/'
                    proxy = 'http://' + prox
                    r = requests.get(url, headers=headers, proxies={'http': proxy, 'https': proxy},  timeout=10)
                    print(r.status_code)
                    if r.status_code in [200, 429]:
                        working_proxies.append(prox)
                        break
                        
                    else:
                        tested_proxies.append(prox)
                except:
                    tested_proxies.append(prox)

    if len(working_proxies) < n_prox:
        time.sleep(20)

    round = round + 1
    

print(f'{len(working_proxies)} proxies found.')

Selecting working proxies.
total fail
total fail
total fail
total fail
429
dldcalksmnc
1 proxies found.


In [9]:
url = 'https://www.idealista.com/'
prox = ''
if prox != '':
    proxy = 'http://' 
else:
    proxy = ''

r = requests.get(url, headers=headers, proxies={'http': proxy, 'https': proxy},  timeout=10)
r.status_code

200

name 'sdf' is not defined
